# Load Dataset

https://huggingface.co/datasets/doc2dial#data-fields


In [1]:
from datasets import load_dataset

split = "train"
cache_dir = "./data_cache"

#User's turn: utterance= question, reference=grounding document span_id, can be empty, "precondition"/"solution" are the
#actual grounding spans
#Gold label for grounding
dialogue_dataset = load_dataset(
    "doc2dial",
    name="dialogue_domain",  # this is the name of the dataset for the second subtask, dialog generation
    split=split,
    ignore_verifications=True,
    cache_dir=cache_dir,
)

document_dataset = load_dataset(
    "doc2dial",
    name="document_domain",
    split=split,
    ignore_verifications=True,
    cache_dir=cache_dir,
)

Reusing dataset doc2dial (./data_cache/doc2dial/dialogue_domain/1.0.1/c15afdf53780a8d6ebea7aec05384432195b356f879aa53a4ee39b740d520642)
Reusing dataset doc2dial (./data_cache/doc2dial/document_domain/1.0.1/c15afdf53780a8d6ebea7aec05384432195b356f879aa53a4ee39b740d520642)


### Investigate what the the data looks like

Have a first look a the data, how much and what kind of data is there

Questions:
1. How many domains and how frequent are those?
2. How many user utterances don't have a span id, have a "precondition"/"solution"?
3. How often is each span from the documents data used in the dialogue data?

In [2]:
#using pd's DF
import pandas as pd
dialogue_full_df = pd.DataFrame(data=dialogue_dataset)
document_full_df = pd.DataFrame(data=document_dataset)

#column names
domain_col = 'domain'
doc_id_col = 'doc_id'
spans_col = 'spans'
id_sp_col = 'id_sp'
text_sp_col = 'text_sp'

Info and Describe for Dialogue and a few manual counts

In [81]:
dialogue_full_df

,dial_id,doc_id,domain,turns
0,9f44c1539efe6f7e79b02eb1b413aa43,Top 5 DMV Mistakes and How to Avoid Them#3_0,dmv,"[{'turn_id': 1, 'role': 'user', 'da': 'query_c..."
1,88ecf840ea87f8c53faff15d4f0bb214,Top 5 DMV Mistakes and How to Avoid Them#3_0,dmv,"[{'turn_id': 1, 'role': 'user', 'da': 'query_c..."
2,3c079944cf36c05b45f668669cc3301b,Top 5 DMV Mistakes and How to Avoid Them#3_0,dmv,"[{'turn_id': 1, 'role': 'user', 'da': 'query_c..."
3,9fa8c68e2f211edad9fb65eb5897d600,Top 5 DMV Mistakes and How to Avoid Them#3_0,dmv,"[{'turn_id': 1, 'role': 'user', 'da': 'query_c..."
4,423bd6a66d14c22f8be69add96679255,Top 5 DMV Mistakes and How to Avoid Them#3_0,dmv,"[{'turn_id': 1, 'role': 'user', 'da': 'query_c..."
...,...,...,...,...
3469,38ff454964b59d69c1fb9e1848c1afe9,Student Loan Repayment | Federal Student Aid#1_0,studentaid,"[{'turn_id': 1, 'role': 'user', 'da': 'query_c..."
3470,4c03dc84fa163070675bdb92970d65d1,Student Loan Repayment | Federal Student Aid#1_0,studentaid,"[{'turn_id': 1, 'role': 'user', 'da': 'query_c..."
3471,f709b7ec749abc04bf305dcf8b88e916,Student Loan Repayment | Federal Student Aid#1_0,studentaid,"[{'turn_id': 1, 'role': 'user', 'da': 'query_c..."
3472,fd4b8614da5cf99f37ae135c18b10881,Student Loan Repayment | Federal Student Aid#1_0,studentaid,"[{'turn_id': 1, 'role': 'user', 'da': 'query_c..."


In [3]:
dialogue_full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3474 entries, 0 to 3473
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dial_id  3474 non-null   object
 1   doc_id   3474 non-null   object
 2   domain   3474 non-null   object
 3   turns    3474 non-null   object
dtypes: object(4)
memory usage: 108.7+ KB


In [4]:
dialogue_full_df.describe(include=object)

,dial_id,doc_id,domain,turns
count,3474,3474,3474,3474
unique,3474,415,4,3474
top,9f44c1539efe6f7e79b02eb1b413aa43,Apply for Retirement Benefits | SSA#1_0,dmv,"[{'turn_id': 1, 'role': 'user', 'da': 'query_c..."
freq,1,16,998,1


In [5]:

dialogue_full_df[domain_col].value_counts()

dmv           998
va            967
ssa           831
studentaid    678
Name: domain, dtype: int64

In [6]:
dialogue_full_df[doc_id_col].value_counts()

Apply for Retirement Benefits | SSA#1_0                                                                       16
Learn About Retirement Benefits | SSA#1_0                                                                     16
Benefits Planner | Social Security Administration#1_0                                                         16
Disability Benefits | Social Security Administration#1_0                                                      16
Supplemental Security Income (SSI) Benefits | Social Security Administration#1_0                              16
                                                                                                              ..
Learn what documents you will need to get a Social Security Card | Social Security Administration#13_0_1       1
Learn what documents you will need to get a Social Security Card | Social Security Administration#12_0_1_2     1
Learn what documents you will need to get a Social Security Card | Social Security Administratio

Insights:
- there are four different domains, all seem to be approx equally used with 'dmv' being the most frequent
- there are 415 different documents, quite a few used only once, most frequent use is 16 for a few documents

Look into how turn: here just the first turn

In [41]:
dialogue_full_df['turns']
first_turn = dialogue_full_df['turns'][0]
normalised_first_turn = pd.json_normalize(first_turn)

,turn_id,role,da,references,utterance
0,1,user,query_condition,"[{'sp_id': '4', 'label': 'precondition'}]","Hello, I forgot o update my address, can you h..."
1,2,agent,respond_solution,"[{'sp_id': '6', 'label': 'solution'}, {'sp_id'...","hi, you have to report any change of address t..."
2,3,user,query_solution,"[{'sp_id': '56', 'label': 'solution'}]",Can I do my DMV transactions online?
3,4,agent,respond_solution,"[{'sp_id': '56', 'label': 'solution'}]","Yes, you can sign up for MyDMV for all the onl..."
4,5,user,query_condition,"[{'sp_id': '48', 'label': 'precondition'}]","Thanks, and in case I forget to bring all of t..."
5,6,agent,respond_solution,"[{'sp_id': '49', 'label': 'solution'}, {'sp_id...",This happens often with our customers so that'...
6,7,user,query_solution,"[{'sp_id': '6', 'label': 'solution'}, {'sp_id'...","Ok, and can you tell me again where should I r..."
7,8,agent,respond_solution,"[{'sp_id': '6', 'label': 'solution'}, {'sp_id'...",Sure. Any change of address must be reported t...
8,9,user,query_condition,"[{'sp_id': '40', 'label': 'precondition'}]",Can you tell me more about Traffic points and ...
9,10,agent,respond_solution,"[{'sp_id': '41', 'label': 'solution'}, {'sp_id...",Traffic points is the system used by DMV to tr...


Looking into reference columns in turns

In [48]:
normalised_first_turn[['references']]
first_references = normalised_first_turn['references'][0]
pd.json_normalize(first_references)

,sp_id,label
0,4,precondition


Info and Describe for document

In [166]:
document_full_df

,domain,doc_id,title,doc_text,spans,doc_html_ts,doc_html_raw
0,ssa,Benefits Planner: Survivors | Planning For You...,Benefits Planner: Survivors | Planning For You...,\n\nBenefits Planner: Survivors | Planning For...,"[{'id_sp': '1', 'tag': 'h2', 'start_sp': 0, 'e...","<main><section><div><h2 sent_id=""1"" text_id=""1...","<main class=""content"" id=""content"" role=""main""..."
1,ssa,Benefits Planner: Survivors | Planning For You...,Benefits Planner: Survivors | Planning For You...,"As you plan for the future , you'll want to th...","[{'id_sp': '1', 'tag': 'u', 'start_sp': 0, 'en...","<article><section><div tag_id=""1""><u sent_id=""...",<article>\n<section>\n<p>As you plan for the f...
2,ssa,Benefits Planner: Disability | How You Apply |...,Benefits Planner: Disability | How You Apply |...,\n\nBenefits Planner: Disability | How You App...,"[{'id_sp': '1', 'tag': 'h2', 'start_sp': 0, 'e...","<main><section><div><h2 sent_id=""1"" text_id=""1...","<main class=""content"" id=""content"" role=""main""..."
3,ssa,Benefits Planner: Disability | How You Apply |...,Benefits Planner: Disability | How You Apply |...,You should apply for disability benefits as so...,"[{'id_sp': '1', 'tag': 'u', 'start_sp': 0, 'en...","<article><section><div tag_id=""1""><u sent_id=""...",<article>\n<section>\n<p>You should apply for ...
4,ssa,Learn what documents you will need to get a So...,Learn what documents you will need to get a So...,\n\nCorrected Card for a Foreign Born U.S. Cit...,"[{'id_sp': '1', 'tag': 'h3', 'start_sp': 0, 'e...","<article><h3 sent_id=""1"" text_id=""1"">Corrected...",<article>\n<h3>Corrected Card for a Foreign Bo...
...,...,...,...,...,...,...,...
483,studentaid,Finding and Applying for Scholarships | Federa...,Finding and Applying for Scholarships | Federa...,\n\nFind and apply for as many scholarships as...,"[{'id_sp': '1', 'tag': 'h1', 'start_sp': 0, 'e...",<section><div><div><div><div><div><div><div><d...,"<section class=""section section-content"" id=""s..."
484,studentaid,Loan Repayment Checklist | Federal Student Aid...,Loan Repayment Checklist | Federal Student Aid#1,\n\nAre you a federal student loan borrower? A...,"[{'id_sp': '1', 'tag': 'h1', 'start_sp': 0, 'e...",<section><div><div><div><div><div><div><div><d...,"<section class=""section section-content"" id=""s..."
485,studentaid,Total and Permanent Disability Discharge | Fed...,Total and Permanent Disability Discharge | Fed...,\n\nIf you re totally and permanently disabled...,"[{'id_sp': '1', 'tag': 'h1', 'start_sp': 0, 'e...",<section><div><div><div><div><div><div><div><d...,"<section class=""section section-content"" id=""s..."
486,studentaid,Avoiding Default | Federal Student Aid#1_0,Avoiding Default | Federal Student Aid#1,\n\nThere are steps you can take to repay your...,"[{'id_sp': '1', 'tag': 'h1', 'start_sp': 0, 'e...",<section><div><div><div><div><div><div><div><d...,"<section class=""section section-content"" id=""s..."


In [7]:
document_full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488 entries, 0 to 487
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   domain        488 non-null    object
 1   doc_id        488 non-null    object
 2   title         488 non-null    object
 3   doc_text      488 non-null    object
 4   spans         488 non-null    object
 5   doc_html_ts   488 non-null    object
 6   doc_html_raw  488 non-null    object
dtypes: object(7)
memory usage: 26.8+ KB


In [8]:
document_full_df.describe(include=object)

,domain,doc_id,title,doc_text,spans,doc_html_ts,doc_html_raw
count,488,488,488,488,488,488,488
unique,4,488,475,475,475,476,476
top,dmv,Benefits Planner: Survivors | Planning For You...,Learn what documents you will need to get a So...,\n\nCorrected Card for a Noncitizen Child \nIf...,"[{'id_sp': '1', 'tag': 'h3', 'start_sp': 0, 'e...","<article><h3 sent_id=""1"" text_id=""1"">Corrected...",<article>\n<h3>Corrected Card for a Noncitizen...
freq,149,1,4,4,4,4,4


In [ ]:
document_full_df[domain_col].value_counts()

In [ ]:
document_full_df["title"].value_counts()

Surprisingly not all document titles and spans are unique. Only 475 are, 13 are duplicated?


Look into the spans json a bit more

In [ ]:

document_full_df[spans_col]
# spans are a json array for each document. Each item in the json array is a span with id_sp being the id of the span

In [ ]:
#looking into one span
one_doc_span = document_full_df[spans_col][0]
# reading a frame into pandas
one_doc_span_df  = pd.DataFrame(one_doc_span)
one_doc_span_df

In [ ]:
one_doc_span_df.describe(include=object)

In [ ]:
one_doc_span_df.describe(include=object)

Each span is unique, in this docment there are 99 spans, the interesting columns are id_sp and text_sp

In [20]:
# want to flatten these spans so we have their json keys as columns
# new dataset with domain, doc_id and from json id_sp, text_sp
# not sure if we should keep this all in one big df or a dv per doc_id?
# first try  to flatten it for one document
document_important_cols_df = document_full_df[[domain_col, doc_id_col, spans_col]]
document_important_cols_df

,domain,doc_id,spans
0,ssa,Benefits Planner: Survivors | Planning For You...,"[{'id_sp': '1', 'tag': 'h2', 'start_sp': 0, 'e..."
1,ssa,Benefits Planner: Survivors | Planning For You...,"[{'id_sp': '1', 'tag': 'u', 'start_sp': 0, 'en..."
2,ssa,Benefits Planner: Disability | How You Apply |...,"[{'id_sp': '1', 'tag': 'h2', 'start_sp': 0, 'e..."
3,ssa,Benefits Planner: Disability | How You Apply |...,"[{'id_sp': '1', 'tag': 'u', 'start_sp': 0, 'en..."
4,ssa,Learn what documents you will need to get a So...,"[{'id_sp': '1', 'tag': 'h3', 'start_sp': 0, 'e..."
...,...,...,...
483,studentaid,Finding and Applying for Scholarships | Federa...,"[{'id_sp': '1', 'tag': 'h1', 'start_sp': 0, 'e..."
484,studentaid,Loan Repayment Checklist | Federal Student Aid...,"[{'id_sp': '1', 'tag': 'h1', 'start_sp': 0, 'e..."
485,studentaid,Total and Permanent Disability Discharge | Fed...,"[{'id_sp': '1', 'tag': 'h1', 'start_sp': 0, 'e..."
486,studentaid,Avoiding Default | Federal Student Aid#1_0,"[{'id_sp': '1', 'tag': 'h1', 'start_sp': 0, 'e..."


In [167]:
first_span = document_full_df[spans_col][0]
normalised_first_span = pd.json_normalize(first_span)
normalised_first_span[[id_sp_col, text_sp_col]]

# row_1_df =document_important_cols_df.iloc[0]
# type(row_1_df)
# pd.json_normalize(row_1_df)
# normalised_df = pd.json_normalize(document_df.to_dict() , meta=[domain_col, doc_id_col, spans_col], record_path='spans')
# type(normalised_df)
# type(document_df.spans)
# document_df.spans[0]


# one_doc_span_df[[id_sp_col, text_sp_col]]
#
# pd.json_normalize(document_important_cols_df.spans)



KeyError: 250

Insights from looking at the data structure:
For training I will need the following data:
From Dialogue:
doc_id - Grounding document
turns.utterance -> Dialogue history
references.sp_id -> ID for the output text_sp

From Documents:
To get output span:
doc_id -> spans -> id_sp -> text_sp column is the actual output span


In [ ]:
## Look at querying the arrow dataset directly instead of using pandas df

In [168]:
# dialogue_dataset.scanner(columns=["doc_id", "turns", "references"]).to_table()
# first_
# row_turns = dialogue_dataset[0]['turns']
# print(first_row_turns)
# first_row_turns[0]['references'][0]['sp_id']
# dialogue_dataset['turns']
# dialogue_dataset[0]['turns']
# type(dialogue_dataset)

# dialogue_dataset

#given a dialog id get
# from dialogue [dial_id, doc_id, turn.turn_id, turn.role, turn.utterance, all turn.references.sp_id]
# for each sp_id from document with doc_id: [doc_id, spans.sp_id, spans.text_sp]
first_dialogue=dialogue_dataset[0]
dial_id = dialogue_dataset['dial_id'][0]
doc_id = first_dialogue["doc_id"]
turns = first_dialogue['turns']
document_df_for_id = document_full_df[document_full_df['doc_id'] == doc_id]
spans_for_doc_df = [pd.json_normalize(span) for span in document_df_for_id['spans']][0]

print(f'Dial id: {dial_id}')
print(f'Doc id: {doc_id}')
for turn in turns:
    turn_id = turn['turn_id']
    role = turn['role']
    utterance = turn['utterance']
    references = turn['references']
    # sp_id are strings, parsed to int for df lookup
    sp_ids = [int(ref['sp_id']) for ref in references]
    print(f'turn id: {turn_id}, role: {role}, sp: {sp_ids}, utterance: {utterance}')

# sp_ids
# spans_for_doc_df[spans_for_doc_df['id_sp'] == 41]
spans_for_doc_df

document_full_df[250]




Dial id: 9f44c1539efe6f7e79b02eb1b413aa43
Doc id: Top 5 DMV Mistakes and How to Avoid Them#3_0
turn id: 1, role: user, sp: [4], utterance: Hello, I forgot o update my address, can you help me with that?
turn id: 2, role: agent, sp: [6, 7], utterance: hi, you have to report any change of address to DMV within 10 days after moving. You should do this both for the address associated with your license and all the addresses associated with all your vehicles.
turn id: 3, role: user, sp: [56], utterance: Can I do my DMV transactions online?
turn id: 4, role: agent, sp: [56], utterance: Yes, you can sign up for MyDMV for all the online transactions needed.
turn id: 5, role: user, sp: [48], utterance: Thanks, and in case I forget to bring all of the documentation needed to the DMV office, what can I do?
turn id: 6, role: agent, sp: [49, 50, 52], utterance: This happens often with our customers so that's why our website and MyDMV are so useful for our customers. Just check if you can make your t

KeyError: 250

# Find the Simplest Model
Task:
- input:
    - dialogue history for current discussion (turns.utterance from dialogue)
    - doc_id for the grounding document to find the span from
- output: the text_sp in documents for a certain doc_id that the agent would need to respond to the user

Training:
- embedding for each utterance from the dialogue history
- embedding of the correct text_sp


From discussion with Edwin:
- use dataset directly

Binary classifier to mark up all the spans as being relevant or not


# Evaluation of Simplest Model

Required output format is a json file of predictions with id, prediction_text and no_answer_probability
https://github.com/doc2dial/sharedtask-dialdoc2021/blob/master/scripts/sample_files/sample_prediction_subtask1.json


# Iteration


- extend for training data set as suggested by dialdoc https://mrqa.github.io/2019/shared